# Smartphones and Watches advisor
In this serie of notebooks we will:
- Load text from a Youtube video playlist
- **Split loaded text into appropriate chunks**
- Create a vector store to load chunk embeddings
- Create a Question Answering chatbot:

    - with different prompts techniques
    - with different similarity measures
    - with or without chat memory 

!['Data Connection'](../../images/les_numeriques_youtube/data_connection.png)


# How to perform a split of documents in chunks ? 

- From a Loader, we perform different splits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports 
# Env var
import os 
import sys
from dotenv import load_dotenv, find_dotenv

# Langchain
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# Save loader
from utils import load_docs_from_jsonl, save_docs_to_jsonl

In [3]:
PATH_NAME_LOADER = './loaded_docs.jsonl'
PATH_NAME_SPLITTER = './splitted_docs.jsonl'

In [4]:
# Env variable
sys.path.append('../..')
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [5]:
pages = load_docs_from_jsonl(PATH_NAME_LOADER)

In [6]:
sample_text = pages[0].page_content
print(sample_text)

Bonjour à tous et bienvenue sur Les Numériques. Alors, comme chaque année, le mois de septembre, c'est réservé aux iPhones. Et 2022, évidemment, ne déroge pas la règle. Cette année, on aura vu débarquer la série 14 avec les iPhones 14 et les iPhones 14 Pro. Alors, dans cette vidéo aujourd'hui, on va plutôt s'intéresser aux versions Pro. On reviendra évidemment sur les 14 classiques lors de la sortie de la variante Plus de cet octobre. Mais bref, trêve de bavardage. Je sais que vous voulez savoir ce que vaut ce dernier iPhone. Eh bien, pour la faire court, c'est excellent. Et en même temps, on n'en attendait pas moins d'Apple. On est là face à un des meilleurs smartphones présents sur le marché. Mais ça, c'est vraiment pour résumer. Dans le détail, on peut parler de cet écran OLED qui est le plus lumineux que nous ayons pu voir sur un smartphone. Il peut monter jusqu'à 1955 nits, là où Apple en promet 2000. A titre de comparaison, une dalle d'ordinateur portable dépasse rarement les 500

## Split by character
[LangChain doc](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/character_text_splitter) 

This is the simplest method. 
This splits based on characters (by default "\n\n") and measure chunk length by number of characters.
- How the text is split: by single character
- How the chunk size is measured: by number of characters

In [7]:
# CharacterTextSplitter
c_splitter = CharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separator = ' '
)

c_splitter.split_text(sample_text)

["Bonjour à tous et bienvenue sur Les Numériques. Alors, comme chaque année, le mois de septembre, c'est réservé aux iPhones. Et 2022, évidemment, ne",
 "déroge pas la règle. Cette année, on aura vu débarquer la série 14 avec les iPhones 14 et les iPhones 14 Pro. Alors, dans cette vidéo aujourd'hui, on",
 "va plutôt s'intéresser aux versions Pro. On reviendra évidemment sur les 14 classiques lors de la sortie de la variante Plus de cet octobre. Mais",
 "bref, trêve de bavardage. Je sais que vous voulez savoir ce que vaut ce dernier iPhone. Eh bien, pour la faire court, c'est excellent. Et en même",
 "temps, on n'en attendait pas moins d'Apple. On est là face à un des meilleurs smartphones présents sur le marché. Mais ça, c'est vraiment pour",
 "résumer. Dans le détail, on peut parler de cet écran OLED qui est le plus lumineux que nous ayons pu voir sur un smartphone. Il peut monter jusqu'à",
 "1955 nits, là où Apple en promet 2000. A titre de comparaison, une dalle d'ordinateur portabl

## Recursive character text splitter
[LangChain doc](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter) 


This text splitter is the recommended one for generic text. It is parameterized by a list of characters. 

It tries to split on them in order until the chunks are small enough. 

The default list is ["\n\n", "\n", " ", ""]. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

- How the text is split: by list of characters
- How the chunk size is measured: by number of characters

In [8]:
# Chunk size to small
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""]
)

r_splitter.split_text(sample_text)

["Bonjour à tous et bienvenue sur Les Numériques. Alors, comme chaque année, le mois de septembre, c'est réservé aux iPhones. Et 2022, évidemment, ne",
 "déroge pas la règle. Cette année, on aura vu débarquer la série 14 avec les iPhones 14 et les iPhones 14 Pro. Alors, dans cette vidéo aujourd'hui, on",
 "va plutôt s'intéresser aux versions Pro. On reviendra évidemment sur les 14 classiques lors de la sortie de la variante Plus de cet octobre. Mais",
 "bref, trêve de bavardage. Je sais que vous voulez savoir ce que vaut ce dernier iPhone. Eh bien, pour la faire court, c'est excellent. Et en même",
 "temps, on n'en attendait pas moins d'Apple. On est là face à un des meilleurs smartphones présents sur le marché. Mais ça, c'est vraiment pour",
 "résumer. Dans le détail, on peut parler de cet écran OLED qui est le plus lumineux que nous ayons pu voir sur un smartphone. Il peut monter jusqu'à",
 "1955 nits, là où Apple en promet 2000. A titre de comparaison, une dalle d'ordinateur portabl

In [9]:
# Chunk size too small, overlap 
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=50, # Add overlap
    separators=["\n\n", "\n", " ", ""]
)

r_splitter.split_text(sample_text)

['Bonjour à tous et bienvenue sur Les Numériques. Alors, comme chaque année, le mois de septembre,',
 "Alors, comme chaque année, le mois de septembre, c'est réservé aux iPhones. Et 2022, évidemment, ne",
 'réservé aux iPhones. Et 2022, évidemment, ne déroge pas la règle. Cette année, on aura vu débarquer',
 'pas la règle. Cette année, on aura vu débarquer la série 14 avec les iPhones 14 et les iPhones 14',
 "la série 14 avec les iPhones 14 et les iPhones 14 Pro. Alors, dans cette vidéo aujourd'hui, on va",
 "Pro. Alors, dans cette vidéo aujourd'hui, on va plutôt s'intéresser aux versions Pro. On reviendra",
 "s'intéresser aux versions Pro. On reviendra évidemment sur les 14 classiques lors de la sortie de",
 'sur les 14 classiques lors de la sortie de la variante Plus de cet octobre. Mais bref, trêve de',
 'variante Plus de cet octobre. Mais bref, trêve de bavardage. Je sais que vous voulez savoir ce que',
 'bavardage. Je sais que vous voulez savoir ce que vaut ce dernier iPhone. Eh b

In [10]:
# Chunk size big enough
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

r_splitter.split_text(sample_text)

["Bonjour à tous et bienvenue sur Les Numériques. Alors, comme chaque année, le mois de septembre, c'est réservé aux iPhones. Et 2022, évidemment, ne déroge pas la règle. Cette année, on aura vu débarquer la série 14 avec les iPhones 14 et les iPhones 14 Pro. Alors, dans cette vidéo aujourd'hui, on va plutôt s'intéresser aux versions Pro. On reviendra évidemment sur les 14 classiques lors de la sortie de la variante Plus de cet octobre. Mais bref, trêve de bavardage.",
 "Je sais que vous voulez savoir ce que vaut ce dernier iPhone. Eh bien, pour la faire court, c'est excellent. Et en même temps, on n'en attendait pas moins d'Apple. On est là face à un des meilleurs smartphones présents sur le marché. Mais ça, c'est vraiment pour résumer. Dans le détail, on peut parler de cet écran OLED qui est le plus lumineux que nous ayons pu voir sur un smartphone. Il peut monter jusqu'à 1955 nits, là où Apple en promet 2000.",
 "A titre de comparaison, une dalle d'ordinateur portable dépasse rareme

In [11]:
docs = r_splitter.split_documents(pages)
save_docs_to_jsonl(docs, PATH_NAME_SPLITTER)